In [ ]:
# import json
# # Get team to member mapping
# with open("team_mapping.json") as file:
# 	TEAM_MEMBERS = json.load(file)

# IAM_USERS = []
# for users in TEAM_MEMBERS.values():
# 	IAM_USERS.extend(users)

In [ ]:
# from aws_user_manager import AwsUserManager
# um = AwsUserManager()
# um.recreate_all_users(TEAM_MEMBERS)

In [ ]:
from src.aws_policy_manager import AWSPolicyManager
pm = AWSPolicyManager(region="us-east-1")

team = "DE_27_Team7"
DOMAIN_GROUPS = {
	"Compute":      ["EC2", "CloudShell"],
	"Data":         ["S3", "RDS", "Glue","DynamoDB"],
	"Integration":  ["EventBridge", "Airflow"],
	"ML":           ["SageMaker","SageMakerAI", "Bedrock"],
	"Monitoring":   ["CloudWatch"],
	"Catalog":      ["ServiceCatalog"],
    "Network":	  	["Policy"]
}

pm.create_team_domain_policies(
    team_name=team,
    DOMAIN_GROUPS=DOMAIN_GROUPS
)
pm.attach_mwaa_group_prereqs_via_policy_maker(team)



⚠ Policy DE_27_Team7-ComputeAccess exists → updating
🧹 Deleting old version: v47
🔄 Updated policy: DE_27_Team7-ComputeAccess
🔗 Attached arn:aws:iam::381492212823:policy/DE_27_Team7-ComputeAccess → DE_27_Team7
📌 Attached DE_27_Team7-ComputeAccess → DE_27_Team7
⚠ Policy DE_27_Team7-DataAccess exists → updating
🧹 Deleting old version: v44
🔄 Updated policy: DE_27_Team7-DataAccess
🔗 Attached arn:aws:iam::381492212823:policy/DE_27_Team7-DataAccess → DE_27_Team7
📌 Attached DE_27_Team7-DataAccess → DE_27_Team7
⚠ Policy DE_27_Team7-IntegrationAccess exists → updating
🧹 Deleting old version: v40
🔄 Updated policy: DE_27_Team7-IntegrationAccess
🔗 Attached arn:aws:iam::381492212823:policy/DE_27_Team7-IntegrationAccess → DE_27_Team7
📌 Attached DE_27_Team7-IntegrationAccess → DE_27_Team7
⚠ Policy DE_27_Team7-MLAccess exists → updating
🧹 Deleting old version: v37
🔄 Updated policy: DE_27_Team7-MLAccess
🔗 Attached arn:aws:iam::381492212823:policy/DE_27_Team7-MLAccess → DE_27_Team7
📌 Attached DE_27_Team7

In [ ]:
from src.aws_infra_manager import AWSInfraManager

infra = AWSInfraManager(region="us-east-1")
team = "DE_27_Team7"

# 1. Create S3 bucket
# bucket = infra.create_team_bucket(team)

# # 2. Create Glue role
# glue_role = infra.create_glue_role(
#     team_name=team,
#     managed_policy_arns=[
#         "arn:aws:iam::aws:policy/service-role/AWSGlueServiceRole"
#     ]
# )

# # 3. Create EC2 role
# ec2_role = infra.create_ec2_role(
#     team_name=team,
#     managed_policy_arns=[
#         "arn:aws:iam::aws:policy/AmazonEC2FullAccess"
#     ]
# )

# # 1. Create MWAA S3 Buckets
mwaa_buckets = infra.create_mwaa_bucket(team)

# # 2. Create MWAA Execution Role
mwaa_role = infra.create_mwaa_execution_role(
    team_name=team
)

infra.update_mwaa_trust_policy(mwaa_role)


print("Buckets:", mwaa_buckets)
print("MWAA Role:", mwaa_role)


✅ Created MWAA bucket: de-27-team7
📁 Created folder: s3://de-27-team7/dags/
📁 Created folder: s3://de-27-team7/plugins/
📁 Created folder: s3://de-27-team7/requirements/
✅ MWAA execution policy attached
✅ MWAA trust policy fixed for de-27-team7-mwaa-exec-role
Buckets: {'bucket': 'de-27-team7', 'dags': 'de-27-team7/dags/', 'plugins': 'de-27-team7/plugins/', 'requirements': 'de-27-team7/requirements/'}
MWAA Role: de-27-team7-mwaa-exec-role


In [3]:
mwaa_role = infra.create_mwaa_execution_role(team)

infra.attach_secrets_manager_access(
    role_name=mwaa_role,
    team_name=team
)


⚠ MWAA role already exists: de-27-team7-mwaa-exec-role
✅ Attached MWAA execution policy
✅ Secrets Manager access attached to role: de-27-team7-mwaa-exec-role


In [ ]:
from src.aws_network_manager import AWSNetworkManager

net = AWSNetworkManager(region="us-east-1")
team = "DE_27_Team7"

vpc = net.get_or_create_shared_vpc()
igw = net.get_or_create_shared_igw(vpc)

pub = net.get_existing_or_create_subnet(net.PUBLIC_SUBNET_CIDR, "public", vpc, public=True)
priv1 = net.get_existing_or_create_subnet(net.PRIVATE_SUBNET_CIDR_1, "private-1", vpc)
priv2 = net.get_existing_or_create_subnet(net.PRIVATE_SUBNET_CIDR_2, "private-2", vpc)

net.get_or_create_rtb(vpc, igw, pub)
net.create_shared_rds_subnet_group(priv1, priv2)

net.create_team_keypair(team)
sg = net.create_team_sg(team, vpc)
net.create_or_update_launch_template(team, pub, sg)

net.attach_team_lt_policy(team)
net.attach_team_rds_policy(team)


🟢 Shared VPC exists: vpc-0f8a2c9945babf1cf
🌐 Shared IGW exists: igw-0ef06d2a1edf1c7ce
🟨 Reusing subnet public: subnet-060d032410b27c135
🟨 Reusing subnet private-1: subnet-06ad0c99e45ca2785
🟨 Reusing subnet private-2: subnet-0e1154b10e02638ec
🔄 Policy exists, rotating: de-27-team7-LTVisibility
🧹 Deleting old version: v4
✅ Updated policy: de-27-team7-LTVisibility
✅ Created policy: de-27-team7-RDSVisibility
